# Fast initial tests to check good ranges and behaviours
# with printout and visuals

In [ ]:
import os

import numpy
from matplotlib import pyplot

from pyhf_stuff import (
    fit_cabinetry,
    fit_cabinetry_post,
    fit_linspace,
    fit_mcmc_mix,
    fit_normal,
    fit_signal,
    mcmc_core,
    region,
)
from pyhf_stuff.region_fit import region_fit

In [ ]:
# fill me in
BASEPATH = "/home/tombs/Cambridge/interval-evidence/searches/ins1771533/"

# fits

In [ ]:
def main_fits():
    region_name_to_scan = {
        "SRlow": (10, 100),
        "SRISR": (10, 40),
    }

    region_name_to_anchors = {
        "SRlow": [10.0],
    }

    for name, (lo, hi) in region_name_to_scan.items():
        print(name)
        dump(name, lo, hi, region_name_to_anchors=region_name_to_anchors)


def dump(name, lo, hi, nbins=10, region_name_to_anchors=None):
    if region_name_to_anchors is None:
        region_name_to_anchors = {}

    dir_region = os.path.join(BASEPATH, name)
    region_1 = region.Region.load(dir_region)

    # cabinetry
    cab = fit_cabinetry.fit(region_1)
    print(cab)
    cab_post = fit_cabinetry_post.fit(region_1)
    print(cab_post)

    # normal
    norm = fit_normal.fit(region_1)
    print(norm)

    # linspace
    linspace = fit_linspace.fit(
        region_1,
        lo,
        hi,
        nbins + 1,
        anchors=region_name_to_anchors.get(name),
    )
    print(linspace)

    levels = numpy.array(linspace.levels) - region_fit(region_1).fun

    x = numpy.linspace(linspace.start, linspace.stop, len(levels))
    pyplot.plot(x, -levels, "k")
    pyplot.ylim(-8, 0)
    pyplot.show()


main_fits()

# mcmc

In [ ]:
def main_mcmc():
    region_name_to_scan = {
        "SRlow": (10, 100),
        "SRISR": (10, 40),
    }

    for name, (lo, hi) in region_name_to_scan.items():
        print(name)
        dump_region(name, lo, hi)


def dump_region(name, lo, hi, nbins=50):
    dir_region = os.path.join(BASEPATH, name)
    region_1 = region.Region.load(dir_region)

    mix = fit_mcmc_mix.fit(
        region_1,
        nbins,
        (lo, hi),
        seed=0,
        nsamples=10_000,
        nrepeats=8,
    )

    neff = mcmc_core.n_by_fit(mix).sum()
    nrepeats = mix.nrepeats
    nsamples = mix.nsamples
    total = numpy.sum(mix.yields)
    print(
        "acceptance: %.2f (%d / %d)"
        % (total / (nrepeats * nsamples), total, nrepeats * nsamples)
    )
    print(
        "efficiency: %.2f (%.1f / %.1f)"
        % (nrepeats * neff / total, neff, total / nrepeats)
    )

    x = numpy.linspace(*mix.range_, len(mix.yields) + 1)
    y = numpy.array(mix.yields)
    pyplot.hist(
        x[:-1],
        weights=y / y.max(),
        range=mix.range_,
        bins=len(x) - 1,
        histtype="step",
        color="r",
        lw=2,
    )

    pyplot.show()
    
main_mcmc()

# signal scan

In [ ]:
def main_signal():
    region_name_to_scan = {
        "SRlow": (0, 60),
        "SRISR": (0, 40),
    }

    for name, (lo, hi) in region_name_to_scan.items():
        print(name)
        dump_region(name, lo, hi)


def dump_region(name, lo, hi, nbins=10):
    dir_region = os.path.join(BASEPATH, name)
    region_1 = region.Region.load(dir_region)

    dir_fit = os.path.join(dir_region, "fit")

    sig = fit_signal.fit(region_1, lo, hi, nbins + 1)
    print(sig)

    levels = numpy.array(sig.levels)
    levels -= levels.min()

    x = numpy.linspace(sig.start, sig.stop, len(levels))
    pyplot.plot(x, levels, "k")
    pyplot.ylim(0, 8)
    pyplot.show()


main_signal()